In [22]:
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from typing import List

In [23]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)

cpu


In [24]:
token_model ='skt/kobart-base-v1'

In [25]:
try:
    tokenizer = PreTrainedTokenizerFast.from_pretrained(token_model)
    model = BartForConditionalGeneration.from_pretrained(token_model).to(device)
except Exception as e:
    print(f"모델 로딩 실패: {e}")
    raise

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [26]:
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_lay

In [37]:
def generate_kobart(prompt: str, max_length=512):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)

    # token_type_ids 제거 (BART 계열엔 필요 없음)
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']

    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 텍스트 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.8,
        eos_token_id=tokenizer.eos_token_id
    )

    # 전체 출력 디코딩
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    
    # 프롬프트를 포함해서 생성되므로, 프롬프트 제거
    if generated.startswith(prompt.strip()):
        return generated[len(prompt.strip()):].strip()
    return generated


In [40]:
def chunk_text(text: str, max_chars: int = 300) -> List[str]:
    return [text[i:i + max_chars] for i in range(0, len(text), max_chars)]

In [44]:
def convert_to_written(spoken_text: str) -> str:
    prompt = f"""
다음 구술체를 문어체로 바꿔 주세요:
"{spoken_text}"
"""
    return generate_kobart(prompt)

In [31]:
def read_text_file(filepath: str) -> str:
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {filepath}")
        return ""

In [45]:
if __name__ == "__main__":
    filepath = 'D:/gap_year/poc-story-field-ai/STORY/LLM/travel_test.txt'
    content = read_text_file(filepath)
    if content:
        chunks = chunk_text(content)
        written_parts = [convert_to_written(chunk) for chunk in chunks]
        written = '\n\n'.join(written_parts)
        print(written)

여행,

함 "김" 그 문화원에서 가기 때문에" 왓프의 그 문화원에서 가기 때문에 굉장히 그 숙소 뿐만 아니라 그 숙소 뿐만 아니라 여행지, 아니면 아니면 아니면 어떤에서, 아니면 아니면 그 숙소 예에서 그 해산에서 그 해산사를 딱 투 보고,어요. 그 숙소 뿐만 아니라 그 숙 소 뿐만 그 숙소 뿐만 아니라 아니면 아니면 아니면 아니면 아니면 아니면 그 그 그 그 그 그 그 그 그 그 딱 투이드어요. 그 대학의, 러시아 대학원의 한국 학생이 그 공부를 하는 사람을 해설하는 있어요. 음. 그 너무 재밌고, 볼만하고. 거기서 어떤 거 들어가 어떤 거냐면 우리가 그 시베리아 그 뭐죠? 그 뭐죠? 그 열차 있 있죠? 횡단 열차. 많 열 어르신 씨가 집에서 타고, 여름이 어디에서 이렇게 축하해서 그렇게 축하해서 어디까지 딱 가요. 거기 그 문화원에서 가기 때문에 그렇게 그 문화원에서 가기 때문에 굉장히 그 숙소 뿐만 아니라 그 숙소 뿐만 아니라 그 숙소 뿐만 아니라 여행지, 아니면 아니면 아니면 아니면 아니면 아니면 그런 여행지, 아니면 그 숙소 뿐만 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 아니면 그 그 그 그 그 그 그 그 그 그 그 그렇게 그렇게 그렇게 있어요. 있어요. 음. 그 그 그 그 그 그 그 그 그 그 그 그 그 뭐

네가한테 가는 거예요. 음. 거 거야. 음. 거 네가 그 시빡 시비를 시빡 시비를 몽골까지 갔 그렇게 그렇게 가는 가는 가는 있어요. 음. 거 뭐라고 그렇게 그렇게 그렇게 그렇게 그렇게 그 그 그 시빡 시비를 시프 시비를 시까지 갔 그래서 그래서 그러니까 그러니까 그러니까 굉장히 그 그 그 여행, 여행, 여행지. 그러니까 그러니까 그러니까 굉장히 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 막 막 막 막 막 막 막 막 막 막 막 막 막 막 올라가서 거기 올라가서 굉장히 그 그 막 올라가서 거기 올라가서 굉장히 불

In [46]:
for i, chunk in enumerate(chunks):
    result = convert_to_written(chunk)
    print(f"\n\n--- Chunk {i+1} ---\n{result}")



--- Chunk 1 ---
- 살던" 여행, 구술체를 문어체로 바꿔 주세요: 뭐라고" 여행, 여행, 여행, 여행, 여행하면 혹시 기억에 남는 여행이라든지든지든지 아니면 엄청 좋 좋 좋았던 여행에 대해서 말씀해 주 주 주 주 주 주 박 박 해 주 주 주 주 주 주 실까 있을까 있을까 있을까요? 있을까요? 네. 여행하면 여행하면 배를 타고, 인고, 인차를 세 배를 가지고 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 그 들어요.요.요.요.요.요.요.요. 거라고 있어요.요.요.요.요.요.요.요.요.요. 네. 네. 그렇게요.요.요.요.요. 그런데 그 그 그 전에 120명 정도 백두산만 간 게 아니라 러시아를 10박 11일, 인도 10박 11일, 뭐 중국, 뭐 뭐 뭐 뭐 뭐 뭐 뭐 뭐 할  뭐라고없이 이번에는 그 그 그 그 울릉도 갔다 왔고요. 그래서 그래서 늘 그 그 그 단체로로 가기 때문에 그 그 그 그 단체로 가기 때문에 그 그 그 그 그 단체로 가기 때문에 문화원, 우리예. 그렇게 그리 그리 그리 그리 그리 그리고,고,고, 인고,고, 인차를 세 배를 세 배를 가지고 그 에 그 에이에 그 에이에 그 에이에 으산. 음. 백두산


--- Chunk 2 ---
"김"" 왓프의 그 문화원에서 가기 때문에 굉장히 그 숙소 뿐만 아니라 그 숙소 뿐만 아니라 여행지, 아니면 아니면 아니면 아니면 그 숙소 가까운에서, 아니면 그 그 그 숙소에서, 그 그 숙소 때문에, 거기서 거기서 거기 거기서 거기서 그 숙소 뿐만 많 그 숙소 뿐만 아니라 그 숙소 뿐만 아니라요. 그걸, 아니면 아니면 아니면 그 숙소에,어요. 제가, 러시아 대학, 러시아 대학원의 한국 학생이 그 공부를 하는 사람을 해설하는 거예요. 음. 그 그 그 그 그 그 그리고, 볼고, 볼만하고. 거기서 어떤 거 들어가 어떤 거냐면 우리가 그 시베리아 그 뭐죠? 그 그어서 있 있죠? 횡단 열차. 그 그 여러분 집에서 그렇게 태어나, 여름이 어디에서 이렇게 축하해서 이렇게 축하해서 어디까지 딱 가